<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prelude" data-toc-modified-id="Prelude-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prelude</a></span></li><li><span><a href="#Utilities-(as-before)" data-toc-modified-id="Utilities-(as-before)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Utilities (as before)</a></span></li><li><span><a href="#Finding-Shortest-Path-(as-before)" data-toc-modified-id="Finding-Shortest-Path-(as-before)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Finding Shortest Path (as before)</a></span></li><li><span><a href="#Finding-Shortest-Delivery-Route-(as-before)" data-toc-modified-id="Finding-Shortest-Delivery-Route-(as-before)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Finding Shortest Delivery Route (as before)</a></span></li><li><span><a href="#Time-Handling" data-toc-modified-id="Time-Handling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Time Handling</a></span></li><li><span><a href="#Class-Recorder" data-toc-modified-id="Class-Recorder-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Class Recorder</a></span></li><li><span><a href="#Class-Parcel" data-toc-modified-id="Class-Parcel-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Class Parcel</a></span></li><li><span><a href="#Class-Customer" data-toc-modified-id="Class-Customer-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Class Customer</a></span></li><li><span><a href="#Class-Driver" data-toc-modified-id="Class-Driver-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Class Driver</a></span></li><li><span><a href="#Class-Delivery-Centre" data-toc-modified-id="Class-Delivery-Centre-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Class Delivery Centre</a></span></li><li><span><a href="#Simulation" data-toc-modified-id="Simulation-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Simulation</a></span><ul class="toc-item"><li><span><a href="#Parameters-from-Specification" data-toc-modified-id="Parameters-from-Specification-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Parameters from Specification</a></span></li><li><span><a href="#Generate-Input-Data" data-toc-modified-id="Generate-Input-Data-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Generate Input Data</a></span></li><li><span><a href="#Simulation-Routine" data-toc-modified-id="Simulation-Routine-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Simulation Routine</a></span></li><li><span><a href="#Model-Verification" data-toc-modified-id="Model-Verification-11.4"><span class="toc-item-num">11.4&nbsp;&nbsp;</span>Model Verification</a></span></li></ul></li></ul></div>

# Prelude

In [1]:
import matplotlib.pyplot as plt
import pulp
import math
import random
import pandas as pd
import numpy as np
import simpy

# Utilities (as before)

## Points and Distances

In [2]:
def dist(p1, p2):
    (x1, y1) = p1
    (x2, y2) = p2
    return int(math.sqrt((x1-x2)**2+(y1-y2)**2))

## PlotMap

In [3]:
def plotMap(G, T=[], P=[], W=None,
            style='r-o', lw=1, ms=3, 
            styleT='go', msT=5,
            styleP='b-o', lwP=3, msP=1,
            stylePT='go', msPT=7,
            styleW='bo', msW=9,
            text=None, grid=False):
    fig = plt.gcf()
    fig.set_size_inches(6, 6)
    V, E = G

    if not grid:
        plt.axis('off')
    plt.plot( [ p[0] for p in V ], [ p[1] for p in V ], 'ro', lw=lw, ms=ms)
    for (p, q) in E:
        plt.plot( [ p[0], q[0] ], [ p[1], q[1] ], 'r-o', lw=lw, ms=ms)
    for t in T:
        plt.plot( [ t[0] ], [ t[1] ], 
                  styleT, ms=msT)
    plt.plot( [ p[0] for p in P ],
              [ p[1] for p in P ], 
              styleP, lw=lwP, ms=msP)
    for p in P:
        if p in T:
            plt.plot( [ p[0] ], [ p[1] ], 
                      stylePT, ms=msPT)
    if W is not None:
        plt.plot( [ W[0] ], [ W[1] ], 
                      styleW, ms=msW)
    if text is not None:
        maxX = max([p[0] for p in V])
        plt.text(0.8*maxX, 0, text)
    if grid:
        plt.grid()
    plt.show()

## Add Targets

In [4]:
def addTargets(M, T):
    V, E = M
    E = E.copy()
    V = V.copy()
    for t in T:
        minD = math.inf
        minE = None
        for e in E:
            P, Q = e
            distT = dist(P, t)+dist(t, Q)-dist(P, Q)
            if distT < minD:
                minD = distT
                minE = e
        P, Q = minE
        E.remove( (P, Q) )
        E.append( (P, t) )
        E.append( (t, Q) )
        V.append(t)
    return V, E

## Generate Warehouse Location

In [5]:
def generateWarehouseLocation(M):
    V, _ = M
    W = random.sample(V, k=1)[0]
    return W

## Time Handling

**Convention:** In this project we measure time in seconds. The simulation will start at 0:00. Time related methods will be added as they are needed.

`timestamp(t)` generates a timestamp string in the form `[dd] hh:mm:ss.d`

In [6]:
    def timestamp(t):
        day = int(t)//(24*3600)
        t = t - day*24*3600
        hour = int(t)//3600
        t = t - hour*3600
        mins = int(t)//60
        t = t - mins*60
        secs = int(t)
        t = int(round((t - secs)*10))
        return f"[{day:2d}] {hour:02d}:{mins:02d}:{secs:02d}.{t:1d}"

In [7]:
timestamp(24*3600*3+17*3600+615.1)

'[ 3] 17:10:15.1'

In [8]:
def nextHour(env, hour):
    beginningOfDay = int(env.now//(24*3600))*24*3600
    timeOfDay = env.now-beginningOfDay
    if hour*3600 > timeOfDay:
        return hour*3600 - timeOfDay
    else:
        return hour*3600 + 24*3600 - timeOfDay

In [9]:
def day(now):
    return int(now//(24*3600))

# Finding Shortest Path (as before)

In [10]:
def dist(p1, p2):
    (x1, y1) = p1
    (x2, y2) = p2
    return int(math.sqrt((x1-x2)**2+(y1-y2)**2))

In [11]:
def pathLength(P):
    return 0 if len(P)<=1 else \
            dist(P[0], P[1])+pathLength(P[1:])

In [12]:
def shortestPath(M, A, B):

    def h(p): 
        return pathLength(p)+dist(p[-1],B)
    
    # candidates C are pairs of the path so far and 
    # the heuristic function of that path, 
    # sorted by the heuristic function, as maintained by
    # insert function
    def insert(C, p):
        hp = h(p)
        c = (p, hp)
        for i in range(len(C)):
            if C[i][1]>hp:
                return C[:i]+[c]+C[i:]
        return C+[c]   
        
    V, E = M
    assert(A in V and B in V)    
    C = insert([], [A])

    while len(C)>0:
        # take the first candidate out of the list of candidates
        path, _ = C[0]
        C = C[1:]
        if path[-1]==B:
            return path
        else:
            for (x, y) in E:
                if path[-1]==x and y not in path:
                    C = insert(C, path+[y])
                elif path[-1]==y and x not in path:
                    C = insert(C, path+[x])
    return None

# Finding Shortest Delivery Route (as before)

## Iterative Integer Programming

In [13]:
def createTables(M, T):

    def reverse(P):
        return [ P[-i] for i in range(1,len(P)+1) ]
    
    def index(x, L):
        for i in range(len(L)):
            if x==L[i]: 
                return i
        return None

    n = len(T)
    d = [ [ math.inf for t in T ] for t in T ]
    p = [ [ None for t in T ] for t in T ]
    for i in range(n):
        d[i][i] = 0
        p[i][i] = [ T[i] ]
    for i in range(n):
        for j in range(n):
            if p[i][j] is None:
                s = shortestPath(M, T[i], T[j])
                d[i][j] = d[j][i] = pathLength(s)
                p[i][j] = s
                p[j][i] = reverse(s)
                for m in range(len(s)-1):
                    smi = index(s[m], T)
                    if smi is None:
                        continue
                    for l in range(m+1, len(s)):
                        sli = index(s[l], T)
                        if sli is None:
                            continue  
                        sub = s[m:l+1]
                        if p[smi][sli] is None:
                            p[smi][sli] = sub
                            p[sli][smi] = reverse(sub)
                            d[smi][sli] = d[sli][smi] = pathLength(sub)
    return d,p               

In [14]:
def roundtrips(x, n):
    
    def isElem(x, l):
        for i in range(len(l)):
            if l[i]==x:
                return True
        return False

    def startpoint(trips):
        for i in range(n):
            for t in trips:
                if isElem(i, t):
                    break
            else:
                return i
    
    def totalLength(trips):
        s=0
        for i in range(0, len(trips)):
            s += len(trips[i])-1
        return s

    trips = []
    while totalLength(trips)<n:
        start = startpoint(trips)
        trip = [ start ]
        i = start
        while len(trip) < n-totalLength(trips):
            for j in range(0, n):
                if pulp.value(x[i][j])==1:
                    trip.append(j)
                    i=j
                    break        
            if pulp.value(x[trip[-1]][start])==1:
                trip.append(start)
                break
        trips.append(trip)
    return sorted(trips, key=lambda t: len(t), reverse=True)

In [15]:
import time

def createLoop(M, T, timing=False):
    
    if timing:
        start_time = time.time()
        last_time = time.time()
        
    D, P = createTables(M, T)   # These are the distances between customers and warehouse only
    
    if timing:
        print(f"createTables:   {time.time()-start_time:6.2f}s")
        last_time = time.time()
        
    n = len(T)

    if n==1:
        return T
        
    # create variables
    x = pulp.LpVariable.dicts("x", ( range(n), range(n) ),
                            lowBound=0, upBound=1, cat=pulp.LpInteger)
    # create problem
    prob = pulp.LpProblem("Loop",pulp.LpMinimize)
    # add objective function
    prob += pulp.lpSum([ D[i][j]*x[i][j] 
                             for i in range(n) for j in range(n) ])
    # add constraints
    constraints=0
    for j in range(n):
        prob += pulp.lpSum([ x[i][j] for i in range(n) if i!=j ]) ==1
    constraints += n
    for i in range(n):
        prob += pulp.lpSum([ x[i][j] for j in range(n) if i!=j ]) ==1
    constraints += n
    for i in range(n):
        for j in range(n):
            if i!=j:
                prob += x[i][j]+x[j][i] <= 1
                constraints += 1
    # initialise solver
    solvers = pulp.listSolvers(onlyAvailable=True)
    solver = pulp.getSolver(solvers[0], msg=0)
    prob.solve(solver)

    if timing:
        print(f"Solver:         {time.time()-last_time:6.2f}s {constraints:6,d} Constraints")
        last_time = time.time()

    trips = roundtrips(x, n)
    while len(trips)>1:
        longest = max([ len(t) for t in trips ])
        for t in trips:
            if len(t)<longest:
                prob += pulp.lpSum([ x[t[i]][t[i+1]] + x[t[i+1]][t[i]]
                                        for i in range(0,len(t)-1) ]) <= len(t)-2
                constraints += 1
            else:
                longest = math.inf
        prob.solve(solver)
        
        if timing:
            print(f"Solver:         {time.time()-last_time:6.2f}s {constraints:6,d} Constraints")
            last_time = time.time()

        trips = roundtrips(x, n)
        
    trip = trips[0]
    loop = []
    for k in range(len(trip)-1):
        sub = P[trip[k]][trip[k+1]]
        loop += sub if len(loop)==0 else sub[1:]
        
    if timing:
        print(f"createLoop:     {time.time()-start_time:6.2f}s")

    return loop

## Heuristic Algorithm

In [16]:
def FW(M):
    
    V, E = M

    n = len(V)
    d = [ [ math.inf for j in range(n) ] for i in range(n) ]
    p = [ [ None for j in range(n) ] for i in range(n) ]

    for (A, B) in E:
        a = V.index(A)
        b = V.index(B)
        d[a][b] = d[b][a] = dist(A, B)
        p[a][b] = [A, B]
        p[b][a] = [B, A]
    
    for i in range(n):
        d[i][i] = 0
        p[i][i] = [V[i]]
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                dk = d[i][k] + d[k][j]
                if d[i][j] > dk:
                    d[i][j] = dk
                    p[i][j] = p[i][k][:-1] + p[k][j]
                    
    return d, p

In [17]:
def createLoopH(M, T, timing=False):
    
    def makeLoop(L):
        loop = []
        for i in range(len(L)-1):
            A = L[i]
            B = L[i+1]
            a = V.index(A)
            b = V.index(B)
            sub = P[a][b]
            loop += sub if len(loop)==0 else sub[1:]
        return loop
        
    if timing:
        start_time = time.time()
        last_time = time.time()
        
    V, E = M
    D, P = FW(M)   # note these are the distances between all vertices in M (and T)
    
    if timing:
        print(f"createTables:   {time.time()-start_time:6.2f}s")
        last_time = time.time()
        
    W = T[0]
    customers = T[1:]
    if len(T)==1:
        L = T
    elif len(T)<=3:
        L = T + [T[0]]
    else:
        L = T[:3]+[T[0]]
        T = T[3:]
        while len(T)>0:
            minExt = math.inf
            minInd = None
            selInd = None
            for k in range(len(T)):
                C = T[k]
                c = V.index(C)
                for i in range(0, len(L)-1):
                    A = L[i]
                    B = L[i+1]
                    a = V.index(A)
                    b = V.index(B)
                    ext = D[a][c] + D[c][b] - D[a][b]
                    if ext<minExt:
                        minExt, minInd, selInd = ext, i+1, k
            L = L[:minInd]+[T[selInd]]+L[minInd:]
            T = T[:selInd]+T[selInd+1:]
            
    if timing:
        print(f"createLoopH:    {time.time()-start_time:6.2f}s")

    return makeLoop(L)

# Class Recorder

We willl use a class Recorder as a reference point for capturing data during the simulation. There will be only one recorder. It will be created at the beginning of every simulation run. Every entity will carry a reference to the Recorder. 

In [18]:
import time

class Recorder:
    
    def __init__(self, env, M, W, C, days, 
                 log=False, plot=False, timing=False):
        self.env = env
        self.M = M
        self.W = W
        self.C = C
        self.days = days
        self.log = log
        self.plot = plot

        self.timing = timing
        self.start_time = time.time()
        self.last_time = self.start_time
        self.cum_timer = {}
            
        Customer.REGISTER = []
        Parcel.REGISTER = []
    
    def timer(self, s):
        t = time.time()
        Δt = t-self.last_time
        if self.timing:
            print(f"==== t: {t-self.start_time:6.2f}s "
                  f"Δt: {Δt:6.2f}s [{s:s}]")
        if s in self.cum_timer:
            self.cum_timer[s] += Δt
        else:
            self.cum_timer[s] = Δt
        self.last_time = t

    def reportTimer(self):
        print()
        for k in sorted(self.cum_timer, key=lambda x: self.cum_timer[x], reverse=True):
            print(f"==== Σ: {self.cum_timer[k]:6.2f}s [{k:s}]")
        print(f"==== -----------------------")
        print(f"==== Σ: {time.time()-self.start_time:6.2f}s Total")
 
    def trace(self, event):
        if self.log:
            print(timestamp(self.env.now), event)

    def finish(self):
        # simulation is finished for good
        # by removing the simulation environment we can
        # pickle recorder
        self.env = None

# Class Parcel

<img src='img/EventGraph Parcel.png' width=500/>

Parcels follow through a sequence of states:
- processing
- in transit (from manufacture to distribution centre)
- arrived in distribution centre
- ready for delivery 
- out for delivery
- customer not present
- returned to distribution centre
- delivered

In [19]:
class Parcel:
    
    REGISTER = []
    
    def __init__(self, rec, i, cust, custIndex):
        self.rec = rec
        self.i = i  # row index in data frames of input data
        self.dest = cust.location
        self.custIndex = custIndex
        self.status = [ 'processing' ] # status record and
        self.timing = [ self.rec.env.now ]      # timing
        assert(len(Parcel.REGISTER)==i)
        Parcel.REGISTER += [ self ] 
    
    # factory method ensures that there is only
    # one Parcel per location
    def getParcel(rec, i, location, custIndex):
        for p in Parcel.REGISTER:
            if p.i == i:
                return p
        return Parcel(rec, i, location, custIndex)

    
    def __str__(self):
        return f"Parcel: {self.i:3d} ({self.custIndex:3d})"

    def index(self):
        return self.i
    
    def destination(self):
        return self.dest
    
    def __reg(self, state):
        self.status += [ state ]
        self.timing += [ self.rec.env.now ]
        self.rec.trace(str(self)+" "+state)
        
    def arrivedAtDeliveryCentre(self):
        self.__reg('arr at delivery centre')
        
    def outForDelivery(self): 
        self.__reg('out for delivery')
        
    def returnFromDelivery(self):
        self.__reg('return from delivery')
    

# Class Customer

<img src='img/EventGraph Customer.png' width=500/>

In [20]:
class Customer:
    
    REGISTER = []

    def __init__(self, rec, location):
        self.rec = rec
        self.location = location
        self.i = len(Customer.REGISTER)
        Customer.REGISTER += [ self ]
        self.atHome = True
        self.answersDoor = False
        self.parcelsReceived = []
        rec.env.process(self.process())
        
    def __str__(self):
        return f"Customer: {self.i:2d} {str(self.location):s}"
    
    # factory method ensures that there is only
    # one customer per location
    def getCustomer(rec, location):
        for c in Customer.REGISTER:
            if c.location == location:
                return c
        return Customer(rec, location)
    
    def leaveHouse(self):
        assert(self.atHome and not self.answersDoor)
        # self.rec.trace(str(self)+" leaves house")
        self.atHome = False
     
    def returnHome(self):
        assert(not self.atHome)
        # self.rec.trace(str(self)+" returns home")
        self.atHome = True
        
    def answerDoor(self):
        if self.atHome:
            yield self.rec.env.timeout(random.expovariate(1/AVERAGE_TIME_ANSWER_DOOR))
            self.rec.trace(str(self)+" answers door")
            self.answersDoor = True
        else:
            yield self.rec.env.timeout(WAIT_TIME_IF_CUSTOMER_DOESNT_ANSWER_DOOR)
            self.rec(str(self)+" not at home")
            
    def acceptParcel(self, parcel):
        assert(self.answersDoor)
        self.parcelsReceived += [parcel]
        self.rec.trace(str(self)+" accepts "+str(parcel))
        
    def signOff(self):
        assert(self.answersDoor)
        self.rec.trace(str(self)+" signs off")
        self.answersDoor = False
        
    def process(self):
        yield self.rec.env.timeout(nextHour(self.rec.env, 8))
        while day(self.rec.env.now)<self.rec.days:
            # in a refinement we may use random times
            self.leaveHouse()
            yield self.rec.env.timeout(nextHour(self.rec.env, 18))
            self.returnHome()
            yield self.rec.env.timeout(nextHour(self.rec.env, 8))

# Class Driver

<img src='img/EventGraph Driver.png' width=500/>

In [21]:
class Driver:
    
    def __init__(self, rec, DC):
        self.rec = rec
        self.DC = DC
        self.location = None
        self.parcels = None
        self.tour = None
        self.rec.env.process(self.process())
        
    # activity
    def __drive(self, target):
        assert(self.tour[0] == self.location)
        while self.location!=target:
            d = dist(self.location, self.tour[1])
            yield self.rec.env.timeout(d / AVERAGE_SPEED)
            self.location = self.tour[1]
            self.tour = self.tour[1:]
        assert(self.tour[0] == self.location == target)
    
    def arriveForWork(self):
        self.location = self.DC.W
        self.parcels = []
        self.returns = []
        self.tour = [ self.DC.W ]
        self.rec.trace("Driver arrives for work")
        
    def leaveForDelivery(self, tour, parcels):
        self.tour, self.parcels = tour, parcels
        self.rec.trace(f"Driver leaves for delivery " \
                       f"of {len(parcels):d} parcels")
        
    def process(self):
        yield self.rec.env.timeout(nextHour(self.rec.env, 18))
        while day(self.rec.env.now)<self.rec.days:
            self.arriveForWork()
            tour, parcels = self.DC.sendForDelivery()
            yield self.rec.env.timeout(PREP_TIME_PER_PARCEL*len(parcels))
            self.leaveForDelivery(tour, parcels)        
            while len(self.parcels)>0:
                # drive to customer
                custLocation = self.parcels[0].dest
                cust = Customer.getCustomer(self.rec, custLocation)
                self.rec.trace("Driver drives to "+str(cust))
                yield from self.__drive(custLocation)
                self.rec.trace("Driver arrived at "+str(cust))
                # call at customer
                yield from cust.answerDoor()
                
                if cust.answersDoor:
                    while len(self.parcels)>0 and \
                            custLocation == self.parcels[0].dest:
                        cust.acceptParcel(self.parcels[0])
                        yield self.rec.env.timeout(random.expovariate(1/10))
                        self.parcels = self.parcels[1:]
                    cust.signOff()
                    yield self.rec.env.timeout(random.expovariate(1/10))
                else:
                    while len(self.parcels)>0 and  \
                            custLocation == self.parcels[0].dest:
                        self.returns += self.parcels[0]
                        self.parcels = self.parcels[1:]

            # return to delivery centre
            self.rec.trace("Driver returns to delivery centre")
            yield from self.__drive(self.DC.W)
            self.rec.trace("Driver arrived at delivery centre")
            
            for parcel in self.returns:
                self.DC.returnFromDelivery(parcel)
                yield self.rec.env.timeout(RETURN_TIME_PER_PARCEL)
            yield self.rec.env.timeout(600)
        
            self.rec.trace("Driver goes home")
            
            yield self.rec.env.timeout(nextHour(self.rec.env, 18))

# Class Delivery Centre

<img src="img/EventGraph Delivery Centre.png" width=300/>

In [22]:
class DeliveryCentre:
    
    def __init__(self, rec, M, W):
        self.rec = rec
        self.M = M
        self.W = W
        self.limit = 35000
        
        self.leftOver = []    # list of parcels
        self.parcels = []     # list of parcels scheduled for delivery
        self.dest = []        # list of unique customer destinations
        self.tour = None      # tour planned for delivery
    
    def __accept(self, parcel):
        custLoc = parcel.dest
        if custLoc not in self.dest:
            MT = addTargets(self.M, self.dest + [custLoc])
            self.rec.timer("addTarget")
            SH = createLoopH(MT, [self.W] + self.dest + [custLoc], 
                             timing=self.rec.timing)
            self.rec.timer("createLoopH")
            if self.tour is None and pathLength(SH)<self.limit:
                self.parcels.append(parcel)
                self.dest += [custLoc]
            else: 
                S = createLoop(MT, [self.W] + self.dest + [custLoc], 
                               timing=self.rec.timing)
                self.rec.timer("createLoop")
                if pathLength(S)<self.limit:
                    self.parcels.append(parcel)
                    self.dest += [custLoc]
                    self.tour = S
                else:
                    self.leftOver.append(parcel)
        else:
            self.parcels.append(parcel)
        
    def acceptParcel(self, parcel):
        parcel.arrivedAtDeliveryCentre()
        self.__accept(parcel)
            
    def sendForDelivery(self):
        parcels = []
        if self.tour is None:
            MT = addTargets(self.M, self.dest)
            self.rec.timer("addTarget")
            self.tour = createLoop(MT, [self.W] + self.dest, 
                                   timing=self.rec.timing)
            self.rec.timer("createLoop")
        tour = self.tour
        addresses = self.dest
        
        # pick parcels in sequence to be delivered
        for i in range(1, len(tour)-1):
            dest = tour[i]
            for p in self.parcels:
                if p.dest == dest and p not in parcels:
                    parcels += [p]
                    p.outForDelivery()
         
        # arrange the left overs
        L = self.leftOver
        self.tour = None
        self.parcels = []
        self.leftOver = []
        self.dest = []
        for p in L:
            self.__accept(p)
        
        if self.rec.plot:
            plotMap(self.rec.M, T=addresses, P=tour, W=tour[0], 
                    text=f"Day {day(self.rec.env.now):2d}, {pathLength(tour):,d}m")

        return tour, parcels
                 
    def returnFromDelivery(self, parcel):
        parcel.returnFromDelivery()
        self.__accept(parcel)

    def getInventory(self):
        return len(self.parcels)+len(self.leftOver)

# Simulation

## Parameters from Specification

The time required for driving is based on the distance between way points at an average speed of 15km/h.

In [23]:
AVERAGE_SPEED = 15/3.6

The **cumulative preparation time** (route planning and 
sorting of the parcels in the delivery order and 
packing the cargo-bike) is assumed to be 50 sec per 
parcel to be delivered.

In [24]:
PREP_TIME_PER_PARCEL = 50

**Additional assumption:** The time to **process returned parcels** in the delivery centre is 30 sec per parce. 

In [25]:
RETURN_TIME_PER_PARCEL = 30

The average time to answer the door.

In [26]:
AVERAGE_TIME_ANSWER_DOOR = 40

In [27]:
WAIT_TIME_IF_CUSTOMER_DOESNT_ANSWER_DOOR = 60

## Generate Input Data

In [28]:
def generateDeliveryData(p, C, days, seed=0):
    ## p is the average number of parcels per day per customer
    ## C is the number of customers to be served
    ## days is the number of days for which data are to be generated.
    np.random.seed(seed)
    R = np.random.poisson(lam=len(C)*p, size=days)
    D = [ sorted(list(np.random.choice(range(len(C)), size=i))) for i in R ]
    return D

In [29]:
def generateInputData(D, log=False):

    R = [ len(d) for d in D ]
    N = sum(R)
    
    DAY_LENGTH = 24*3600    # measured in minutes
    DAY_START = 8*3600      # first delivery in the morning
    DAY_END = 17*3600     # last delivery during day time

    x = pd.DataFrame()

    x['iarr'] = [None]*N
    x['time'] = [None]*N
    x['day'] = [None]*N
    x['dest'] = [None]*N

    current_day = 0
    last_time = 0
    i = 0

    for d in D: # for each day

        if log:
            print("generating for day: ",current_day, D[current_day])
            
        time = current_day*DAY_LENGTH + DAY_START 

        for c in d: # for each customer that should get a 

            IARR = (DAY_END-DAY_START-2*3600) / len(d)  # estimated average IAT for the current day

            iat = random.expovariate(1.0/IARR)
            new_time = time + iat

            x.at[i, 'iarr'] = round(new_time - last_time,1)
            x.at[i, 'time'] = round(new_time - current_day*DAY_LENGTH , 1)
            x.at[i, 'day'] = current_day
            x.at[i, 'dest'] = c
            
            i += 1
            last_time = time = new_time
    
        current_day += 1
    
    return x

## Simulation Routine

In [30]:
def simulation(M, W, C, p=0.2, days=10, seed=0, 
               log=False, plot=False, timing=False):

    random.seed(seed)
    D = generateDeliveryData(p, C, days, seed)
    X = generateInputData(D, log=log)
    
    env = simpy.Environment()
    rec = Recorder(env, M, W, C, days, 
                   log=log, plot=plot, timing=timing)

    print(f"Simulating delivery of {len(X):d} parcels "
          f"over {len(D):d} days to {len(C):d} customers")

    for c in C:
        Customer.getCustomer(rec, c)           
    DC = DeliveryCentre(rec, M, W)
    D = Driver(rec, DC)

    def generatorProcess(env):
        # generate the parcels based on input data x
        for i in range(len(X)):
            yield env.timeout(X.at[i, 'iarr'])
            custIndex = X.at[i, 'dest']
            custLoc = C[custIndex]
            cust = Customer.getCustomer(rec, custLoc)
            p = Parcel.getParcel(rec, i, cust, custIndex)
            DC.acceptParcel(p)

    env.process(generatorProcess(env))
    env.run()

    rec.finish()

    if log:
        print(f"Delivery Centre Inventory: {DC.getInventory():d} parcels")

    return rec

## Model Verification

In [31]:
import pickle
with open('myData.pickled', 'rb') as f:
    M, C = pickle.load(f)

In [33]:
W=generateWarehouseLocation(M)

In [34]:
rec1 = simulation(M, W, C, p=0.15, days=20,seed=6077, log=True, timing=True)
rec1.reportTimer()

generating for day:  0 [10, 13, 16, 18, 31, 33, 33, 41, 43, 44, 46, 51, 55, 68]
generating for day:  1 [0, 7, 11, 11, 26, 41, 45, 48, 49, 54, 55]
generating for day:  2 [7, 7, 8, 20, 29, 31, 32, 33, 34, 39, 40, 42, 61, 69]
generating for day:  3 [18, 26, 27, 30, 45, 52, 67]
generating for day:  4 [3, 22, 37, 46, 50, 60]
generating for day:  5 [17, 32, 39, 49, 55, 55]
generating for day:  6 [1, 9, 19, 28, 39, 46]
generating for day:  7 [1, 6, 8, 12, 13, 15, 29, 34, 35, 38, 58, 67, 68]
generating for day:  8 [4, 10, 13, 29, 42, 46, 56, 60, 61]
generating for day:  9 [14, 15, 23, 26, 35, 38, 42, 49, 52, 53, 57]
generating for day:  10 [0, 3, 6, 18, 18, 24, 25, 32, 34]
generating for day:  11 [13, 14, 17, 26, 27, 29, 31, 45, 47, 52, 69]
generating for day:  12 [7, 9, 10, 12, 20, 29, 31, 41, 58, 63]
generating for day:  13 [12, 18, 19, 26, 26, 51, 58]
generating for day:  14 [17, 22, 54, 56, 66]
generating for day:  15 [8, 11, 12, 29, 30, 30, 36, 45, 56]
generating for day:  16 [6, 14, 15, 

createTables:     0.24s
createLoopH:      0.24s
==== t:   7.34s Δt:   0.24s [createLoopH]
[ 1] 08:34:41.1 Parcel:  15 (  7) arr at delivery centre
==== t:   7.34s Δt:   0.00s [addTarget]
createTables:     0.25s
createLoopH:      0.25s
==== t:   7.60s Δt:   0.25s [createLoopH]
[ 1] 10:03:44.2 Parcel:  16 ( 11) arr at delivery centre
==== t:   7.60s Δt:   0.00s [addTarget]
createTables:     0.31s
createLoopH:      0.31s
==== t:   7.91s Δt:   0.31s [createLoopH]
[ 1] 10:04:56.6 Parcel:  17 ( 11) arr at delivery centre
[ 1] 10:17:05.8 Parcel:  18 ( 26) arr at delivery centre
==== t:   7.91s Δt:   0.00s [addTarget]
createTables:     0.27s
createLoopH:      0.27s
==== t:   8.18s Δt:   0.27s [createLoopH]
[ 1] 10:47:14.8 Parcel:  19 ( 41) arr at delivery centre
==== t:   8.18s Δt:   0.00s [addTarget]
createTables:     0.24s
createLoopH:      0.24s
==== t:   8.42s Δt:   0.24s [createLoopH]
[ 1] 11:12:28.4 Parcel:  20 ( 45) arr at delivery centre
==== t:   8.42s Δt:   0.00s [addTarget]
createTa

createTables:     0.20s
Solver:           0.09s    210 Constraints
createLoop:       0.28s
==== t:  14.36s Δt:   0.28s [createLoop]
[ 2] 18:00:00.0 Parcel:  25 (  7) out for delivery
[ 2] 18:00:00.0 Parcel:  26 (  7) out for delivery
[ 2] 18:00:00.0 Parcel:  33 ( 34) out for delivery
[ 2] 18:00:00.0 Parcel:  35 ( 40) out for delivery
[ 2] 18:00:00.0 Parcel:  34 ( 39) out for delivery
[ 2] 18:00:00.0 Parcel:  37 ( 61) out for delivery
[ 2] 18:00:00.0 Parcel:  38 ( 69) out for delivery
[ 2] 18:00:00.0 Parcel:  36 ( 42) out for delivery
[ 2] 18:00:00.0 Parcel:  31 ( 32) out for delivery
[ 2] 18:00:00.0 Parcel:  30 ( 31) out for delivery
[ 2] 18:00:00.0 Parcel:  32 ( 33) out for delivery
[ 2] 18:00:00.0 Parcel:  28 ( 20) out for delivery
[ 2] 18:00:00.0 Parcel:  27 (  8) out for delivery
[ 2] 18:00:00.0 Parcel:  29 ( 29) out for delivery
[ 2] 18:11:40.0 Driver leaves for delivery of 14 parcels
[ 2] 18:11:40.0 Driver drives to Customer:  7 (2083, 876)
[ 2] 18:21:08.3 Driver arrived at Custo

createTables:     0.35s
createLoopH:      0.35s
==== t:  16.36s Δt:   0.35s [createLoopH]
[ 4] 10:09:10.8 Parcel:  47 ( 22) arr at delivery centre
==== t:  16.36s Δt:   0.00s [addTarget]
createTables:     0.30s
createLoopH:      0.30s
==== t:  16.66s Δt:   0.30s [createLoopH]
[ 4] 10:46:35.6 Parcel:  48 ( 37) arr at delivery centre
==== t:  16.66s Δt:   0.00s [addTarget]
createTables:     0.22s
createLoopH:      0.22s
==== t:  16.89s Δt:   0.22s [createLoopH]
[ 4] 11:35:57.9 Parcel:  49 ( 46) arr at delivery centre
==== t:  16.89s Δt:   0.00s [addTarget]
createTables:     0.21s
createLoopH:      0.21s
==== t:  17.10s Δt:   0.21s [createLoopH]
[ 4] 11:43:09.5 Parcel:  50 ( 50) arr at delivery centre
==== t:  17.10s Δt:   0.00s [addTarget]
createTables:     0.25s
createLoopH:      0.25s
==== t:  17.36s Δt:   0.25s [createLoopH]
[ 4] 12:54:49.6 Parcel:  51 ( 60) arr at delivery centre
==== t:  17.36s Δt:   0.00s [addTarget]
createTables:     0.28s
createLoopH:      0.28s
==== t:  17.64s Δ

createTables:     0.19s
createLoopH:      0.19s
==== t:  20.73s Δt:   0.19s [createLoopH]
[ 7] 09:56:02.3 Parcel:  65 (  6) arr at delivery centre
==== t:  20.73s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t:  20.91s Δt:   0.18s [createLoopH]
[ 7] 09:56:38.2 Parcel:  66 (  8) arr at delivery centre
==== t:  20.91s Δt:   0.00s [addTarget]
createTables:     0.20s
createLoopH:      0.20s
==== t:  21.11s Δt:   0.20s [createLoopH]
[ 7] 10:46:45.4 Parcel:  67 ( 12) arr at delivery centre
==== t:  21.11s Δt:   0.00s [addTarget]
createTables:     0.19s
createLoopH:      0.19s
==== t:  21.30s Δt:   0.19s [createLoopH]
[ 7] 11:29:13.3 Parcel:  68 ( 13) arr at delivery centre
==== t:  21.30s Δt:   0.00s [addTarget]
createTables:     0.19s
createLoopH:      0.19s
==== t:  21.49s Δt:   0.19s [createLoopH]
[ 7] 11:45:48.3 Parcel:  69 ( 15) arr at delivery centre
==== t:  21.49s Δt:   0.00s [addTarget]
createTables:     0.21s
createLoopH:      0.21s
==== t:  21.70s Δ

...

createTables:     0.20s
createLoopH:      0.20s
==== t:  29.15s Δt:   0.20s [createLoopH]
[10] 08:53:23.1 Parcel:  98 (  3) arr at delivery centre
==== t:  29.15s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t:  29.33s Δt:   0.18s [createLoopH]
[10] 10:06:54.8 Parcel:  99 (  6) arr at delivery centre
==== t:  29.33s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t:  29.51s Δt:   0.19s [createLoopH]
[10] 10:54:19.6 Parcel: 100 ( 18) arr at delivery centre
==== t:  29.52s Δt:   0.00s [addTarget]
createTables:     0.17s
createLoopH:      0.17s
==== t:  29.69s Δt:   0.18s [createLoopH]
[10] 11:21:51.2 Parcel: 101 ( 18) arr at delivery centre
[10] 12:11:48.6 Parcel: 102 ( 24) arr at delivery centre
==== t:  29.69s Δt:   0.00s [addTarget]
createTables:     0.19s
createLoopH:      0.19s
==== t:  29.88s Δt:   0.19s [createLoopH]
[10] 12:42:03.7 Parcel: 103 ( 25) arr at delivery centre
==== t:  29.88s Δt:   0.00s [addTarget]
createTa

createTables:     0.19s
createLoopH:      0.19s
==== t:  35.07s Δt:   0.19s [createLoopH]
[12] 08:13:47.7 Parcel: 118 (  9) arr at delivery centre
==== t:  35.08s Δt:   0.00s [addTarget]
createTables:     0.19s
createLoopH:      0.19s
==== t:  35.27s Δt:   0.20s [createLoopH]
[12] 09:15:41.8 Parcel: 119 ( 10) arr at delivery centre
==== t:  35.27s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t:  35.46s Δt:   0.18s [createLoopH]
[12] 09:58:57.8 Parcel: 120 ( 12) arr at delivery centre
==== t:  35.46s Δt:   0.00s [addTarget]
createTables:     0.20s
createLoopH:      0.20s
==== t:  35.66s Δt:   0.20s [createLoopH]
[12] 10:40:52.4 Parcel: 121 ( 20) arr at delivery centre
==== t:  35.66s Δt:   0.00s [addTarget]
createTables:     0.19s
createLoopH:      0.19s
==== t:  35.86s Δt:   0.19s [createLoopH]
[12] 11:10:05.2 Parcel: 122 ( 29) arr at delivery centre
==== t:  35.86s Δt:   0.00s [addTarget]
createTables:     0.28s
createLoopH:      0.28s
==== t:  36.14s Δ

createTables:     0.20s
createLoopH:      0.20s
==== t:  39.40s Δt:   0.20s [createLoopH]
[14] 11:28:04.0 Parcel: 135 ( 22) arr at delivery centre
==== t:  39.40s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t:  39.58s Δt:   0.18s [createLoopH]
[14] 11:56:31.7 Parcel: 136 ( 54) arr at delivery centre
==== t:  39.58s Δt:   0.00s [addTarget]
createTables:     0.20s
createLoopH:      0.20s
==== t:  39.78s Δt:   0.20s [createLoopH]
[14] 12:30:54.6 Parcel: 137 ( 56) arr at delivery centre
==== t:  39.78s Δt:   0.00s [addTarget]
createTables:     0.19s
createLoopH:      0.19s
==== t:  39.97s Δt:   0.19s [createLoopH]
[14] 14:08:06.4 Parcel: 138 ( 66) arr at delivery centre
==== t:  39.97s Δt:   0.00s [addTarget]
createTables:     0.22s
createLoopH:      0.22s
==== t:  40.19s Δt:   0.22s [createLoopH]
[14] 18:00:00.0 Driver arrives for work
==== t:  40.19s Δt:   0.00s [addTarget]
createTables:     0.18s
Solver:           0.07s     42 Constraints
Solver:        

createTables:     0.24s
createLoopH:      0.24s
==== t:  42.76s Δt:   0.24s [createLoopH]
[16] 10:11:47.0 Parcel: 151 ( 21) arr at delivery centre
==== t:  42.76s Δt:   0.00s [addTarget]
createTables:     0.19s
createLoopH:      0.19s
==== t:  42.95s Δt:   0.19s [createLoopH]
[16] 10:18:53.6 Parcel: 152 ( 25) arr at delivery centre
==== t:  42.95s Δt:   0.00s [addTarget]
createTables:     0.20s
createLoopH:      0.20s
==== t:  43.15s Δt:   0.20s [createLoopH]
[16] 10:59:36.7 Parcel: 153 ( 41) arr at delivery centre
==== t:  43.15s Δt:   0.00s [addTarget]
createTables:     0.19s
createLoopH:      0.19s
==== t:  43.35s Δt:   0.19s [createLoopH]
[16] 11:02:42.6 Parcel: 154 ( 42) arr at delivery centre
==== t:  43.35s Δt:   0.00s [addTarget]
createTables:     0.21s
createLoopH:      0.21s
==== t:  43.55s Δt:   0.21s [createLoopH]
[16] 11:05:35.8 Parcel: 155 ( 45) arr at delivery centre
==== t:  43.56s Δt:   0.00s [addTarget]
createTables:     0.21s
createLoopH:      0.21s
==== t:  43.77s Δ

createTables:     0.19s
createLoopH:      0.19s
==== t:  47.63s Δt:   0.19s [createLoopH]
[18] 08:41:09.1 Parcel: 171 ( 10) arr at delivery centre
==== t:  47.63s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t:  47.81s Δt:   0.18s [createLoopH]
[18] 09:15:37.2 Parcel: 172 ( 13) arr at delivery centre
==== t:  47.81s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t:  47.99s Δt:   0.18s [createLoopH]
[18] 09:30:33.3 Parcel: 173 ( 29) arr at delivery centre
==== t:  47.99s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t:  48.17s Δt:   0.18s [createLoopH]
[18] 10:01:38.1 Parcel: 174 ( 53) arr at delivery centre
==== t:  48.17s Δt:   0.00s [addTarget]
createTables:     0.22s
createLoopH:      0.22s
==== t:  48.39s Δt:   0.22s [createLoopH]
[18] 10:07:51.1 Parcel: 175 ( 57) arr at delivery centre
==== t:  48.39s Δt:   0.00s [addTarget]
createTables:     0.21s
createLoopH:      0.21s
==== t:  48.60s Δ

In [35]:
rec3 = simulation(M, W, C, p=0.15, days=20,seed=6077)
rec3.reportTimer()

Simulating delivery of 189 parcels over 20 days to 70 customers

==== Σ:  42.76s [createLoopH]
==== Σ:  11.60s [createLoop]
==== Σ:   0.41s [addTarget]
==== -----------------------
==== Σ:  54.77s Total
